# Importations

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
import numpy as np
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import cross_val_score

In [2]:
data = pd.read_csv("../raw_data/data_final_indé_small.csv")

## Présentation de la donnée

In [3]:
data

,steam_appid,name,top_5_tags,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,...,english,price,has_a_website,nb_review,owner_estimated,rating,popularity,score_descriptif,average_playtime,median_playtime
0,4290,RACE: Caterham Expansion,racing simulation relaxing realistic real_time...,0.015822,0.015822,0.015822,0.015822,0.015822,0.015822,0.015822,...,1,3.99,1,6,1248.0,424.233333,517.758333,4,0,0
1,4900,Zen of Sudoku,casual puzzle 1980s relaxing realistic,0.015666,0.307211,0.015666,0.015666,0.015666,0.015666,0.015666,...,1,0.00,1,139,20000.0,22124.365000,23631.392500,3,0,0
2,7340,Azada,casual puzzle 1980s relaxing realistic,0.015666,0.307211,0.015666,0.015666,0.015666,0.015666,0.015666,...,1,6.99,1,12,2496.0,7.012399,9.197810,1,0,0
3,9960,Haunted House™,adventure 1980s relaxing realistic real_time_w...,0.015546,0.015546,0.015546,0.015546,0.015546,0.015546,0.015546,...,1,4.79,1,22,4576.0,215.087500,263.378125,3,0,0
4,9970,Star Raiders,action space 1980s realistic real_time_with_pause,0.016158,0.016158,0.016158,0.016158,0.016158,0.016158,0.016158,...,1,6.99,1,29,6032.0,209.787500,263.028125,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18524,1065230,Room of Pandora,adventure casual puzzle 1980s relaxing,0.015694,0.701810,0.015694,0.015694,0.015694,0.015694,0.015694,...,1,2.09,0,3,624.0,3.000000,3.000000,1,0,0
18525,1065570,Cyber Gun,action adventure fps 3d_platformer fast_paced,0.016478,0.016478,0.519926,0.016478,0.183470,0.016478,0.016478,...,1,1.69,0,9,1872.0,13.366667,16.816667,3,0,0
18526,1065650,Super Star Blast,action casual 1980s relaxing realistic,0.015483,0.015483,0.015483,0.015483,0.015483,0.015483,0.015483,...,1,3.99,1,1,208.0,4.600000,10.450000,5,0,0
18527,1066700,New Yankee 7: Deer Hunters,casual adventure 1980s relaxing realistic,0.015466,0.015466,0.015466,0.015466,0.015466,0.015466,0.015466,...,1,5.19,1,2,416.0,23.463964,29.301802,2,0,0


## Sélection de X et Y

In [4]:
X = data.drop(columns=["steam_appid",
                        "name",
                       "top_5_tags",
                       "nb_review",
                       "owner_estimated",
                       "rating",
                       "topic_7",
                       "topic_11",
                       "popularity",
                      "average_playtime",
                    "median_playtime"])

y = data['owner_estimated']

## Train test split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Scale de la donnée

In [6]:
scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train)
scaler.transform(X_test)

array([[-0.28861779, -0.25262812, -0.25083665, ...,  0.00498739,
         0.80253433,  1.4810919 ],
       [-0.28880734, -0.25287351, -0.25106148, ...,  0.6131767 ,
        -1.24605261,  0.70198085],
       [-0.28744769, -0.25111336, -0.24944878, ..., -0.65849185,
        -1.24605261, -0.07713019],
       ...,
       [-0.28198992, -0.24404789, -0.24297522, ..., -0.53408949,
        -1.24605261,  0.31242533],
       [-0.28915744, -0.25332673, -0.25147674, ...,  2.41009964,
         0.80253433, -0.46668571],
       [-0.29054917, -0.25512843, -0.2531275 , ...,  0.75140154,
         0.80253433, -0.07713019]])

## Entrainement du modèle avec Catboost

In [7]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(custom_metric=['RMSE','R2'])
# Fit model
model.fit(X_train, y_train)
# Get predictions
preds = model.predict(X_test)

Learning rate set to 0.06138
0:	learn: 6297.4665022	total: 61ms	remaining: 1m
1:	learn: 6251.1235167	total: 63.8ms	remaining: 31.8s
2:	learn: 6209.9607147	total: 66.4ms	remaining: 22.1s
3:	learn: 6168.4693673	total: 68.7ms	remaining: 17.1s
4:	learn: 6134.5717157	total: 71ms	remaining: 14.1s
5:	learn: 6101.0595912	total: 73.4ms	remaining: 12.2s
6:	learn: 6070.5810388	total: 75.7ms	remaining: 10.7s
7:	learn: 6044.4797000	total: 78.2ms	remaining: 9.69s
8:	learn: 6019.4230880	total: 80.2ms	remaining: 8.84s
9:	learn: 6000.1576138	total: 82.8ms	remaining: 8.2s
10:	learn: 5978.9895762	total: 85.4ms	remaining: 7.68s
11:	learn: 5955.7379642	total: 87.8ms	remaining: 7.22s
12:	learn: 5937.9199704	total: 90.1ms	remaining: 6.84s
13:	learn: 5921.1820425	total: 92.9ms	remaining: 6.54s
14:	learn: 5906.4662959	total: 95ms	remaining: 6.24s
15:	learn: 5890.5422178	total: 97.1ms	remaining: 5.97s
16:	learn: 5876.9171295	total: 100ms	remaining: 5.79s
17:	learn: 5864.7206125	total: 102ms	remaining: 5.59s
18:

212:	learn: 5371.8842602	total: 585ms	remaining: 2.16s
213:	learn: 5369.6058127	total: 587ms	remaining: 2.15s
214:	learn: 5368.6096765	total: 589ms	remaining: 2.15s
215:	learn: 5367.0563847	total: 591ms	remaining: 2.15s
216:	learn: 5365.6694486	total: 593ms	remaining: 2.14s
217:	learn: 5363.7101445	total: 596ms	remaining: 2.14s
218:	learn: 5361.7826955	total: 599ms	remaining: 2.13s
219:	learn: 5360.6797905	total: 601ms	remaining: 2.13s
220:	learn: 5358.9685874	total: 604ms	remaining: 2.13s
221:	learn: 5357.7957139	total: 606ms	remaining: 2.12s
222:	learn: 5356.4566229	total: 608ms	remaining: 2.12s
223:	learn: 5355.5885867	total: 611ms	remaining: 2.12s
224:	learn: 5354.1223831	total: 613ms	remaining: 2.11s
225:	learn: 5352.5737937	total: 616ms	remaining: 2.11s
226:	learn: 5351.7106807	total: 618ms	remaining: 2.1s
227:	learn: 5349.8365030	total: 620ms	remaining: 2.1s
228:	learn: 5348.5076536	total: 623ms	remaining: 2.1s
229:	learn: 5346.6335780	total: 625ms	remaining: 2.09s
230:	learn: 5

376:	learn: 5163.0868424	total: 976ms	remaining: 1.61s
377:	learn: 5161.7768794	total: 978ms	remaining: 1.61s
378:	learn: 5160.2585864	total: 981ms	remaining: 1.61s
379:	learn: 5159.4603212	total: 983ms	remaining: 1.6s
380:	learn: 5158.9201887	total: 985ms	remaining: 1.6s
381:	learn: 5158.1448554	total: 988ms	remaining: 1.6s
382:	learn: 5157.0523214	total: 991ms	remaining: 1.59s
383:	learn: 5156.5875104	total: 993ms	remaining: 1.59s
384:	learn: 5155.4432792	total: 995ms	remaining: 1.59s
385:	learn: 5153.9630537	total: 998ms	remaining: 1.59s
386:	learn: 5152.7535778	total: 1s	remaining: 1.58s
387:	learn: 5151.7270995	total: 1s	remaining: 1.58s
388:	learn: 5150.0575144	total: 1s	remaining: 1.58s
389:	learn: 5149.0868358	total: 1.01s	remaining: 1.57s
390:	learn: 5148.1576245	total: 1.01s	remaining: 1.57s
391:	learn: 5147.5401778	total: 1.01s	remaining: 1.57s
392:	learn: 5145.9212542	total: 1.01s	remaining: 1.57s
393:	learn: 5144.8317329	total: 1.02s	remaining: 1.56s
394:	learn: 5143.18801

536:	learn: 5003.9697723	total: 1.38s	remaining: 1.19s
537:	learn: 5003.0614330	total: 1.38s	remaining: 1.18s
538:	learn: 5002.3652427	total: 1.38s	remaining: 1.18s
539:	learn: 5001.5641122	total: 1.38s	remaining: 1.18s
540:	learn: 5000.8744991	total: 1.39s	remaining: 1.18s
541:	learn: 4999.8669823	total: 1.39s	remaining: 1.17s
542:	learn: 4999.4982125	total: 1.39s	remaining: 1.17s
543:	learn: 4998.7994466	total: 1.39s	remaining: 1.17s
544:	learn: 4998.1226796	total: 1.4s	remaining: 1.17s
545:	learn: 4997.2604081	total: 1.4s	remaining: 1.16s
546:	learn: 4996.5945621	total: 1.4s	remaining: 1.16s
547:	learn: 4995.7222429	total: 1.4s	remaining: 1.16s
548:	learn: 4995.1161699	total: 1.41s	remaining: 1.15s
549:	learn: 4993.6616038	total: 1.41s	remaining: 1.15s
550:	learn: 4993.2838750	total: 1.41s	remaining: 1.15s
551:	learn: 4992.3733049	total: 1.41s	remaining: 1.15s
552:	learn: 4991.2815143	total: 1.41s	remaining: 1.14s
553:	learn: 4990.3009875	total: 1.42s	remaining: 1.14s
554:	learn: 49

699:	learn: 4862.8410948	total: 1.77s	remaining: 757ms
700:	learn: 4862.2440692	total: 1.77s	remaining: 755ms
701:	learn: 4861.6275298	total: 1.77s	remaining: 752ms
702:	learn: 4860.6550614	total: 1.77s	remaining: 749ms
703:	learn: 4860.0234141	total: 1.78s	remaining: 747ms
704:	learn: 4859.3120798	total: 1.78s	remaining: 744ms
705:	learn: 4858.1655273	total: 1.78s	remaining: 742ms
706:	learn: 4857.7227152	total: 1.78s	remaining: 739ms
707:	learn: 4856.8944170	total: 1.78s	remaining: 737ms
708:	learn: 4855.7703909	total: 1.79s	remaining: 734ms
709:	learn: 4854.8222264	total: 1.79s	remaining: 732ms
710:	learn: 4854.4089089	total: 1.79s	remaining: 729ms
711:	learn: 4853.4247221	total: 1.79s	remaining: 726ms
712:	learn: 4852.8828301	total: 1.8s	remaining: 724ms
713:	learn: 4851.4913700	total: 1.8s	remaining: 721ms
714:	learn: 4850.6848173	total: 1.8s	remaining: 719ms
715:	learn: 4850.2237235	total: 1.8s	remaining: 716ms
716:	learn: 4849.2617788	total: 1.81s	remaining: 714ms
717:	learn: 48

863:	learn: 4738.8719753	total: 2.16s	remaining: 340ms
864:	learn: 4737.7233829	total: 2.16s	remaining: 337ms
865:	learn: 4737.0987832	total: 2.16s	remaining: 335ms
866:	learn: 4736.0053245	total: 2.17s	remaining: 332ms
867:	learn: 4734.9918491	total: 2.17s	remaining: 330ms
868:	learn: 4734.1573730	total: 2.17s	remaining: 327ms
869:	learn: 4733.1350459	total: 2.17s	remaining: 325ms
870:	learn: 4732.4538502	total: 2.17s	remaining: 322ms
871:	learn: 4731.4822230	total: 2.18s	remaining: 320ms
872:	learn: 4730.5347545	total: 2.18s	remaining: 317ms
873:	learn: 4730.0178694	total: 2.18s	remaining: 315ms
874:	learn: 4729.6246927	total: 2.18s	remaining: 312ms
875:	learn: 4728.8885721	total: 2.19s	remaining: 310ms
876:	learn: 4728.4753895	total: 2.19s	remaining: 307ms
877:	learn: 4727.4550849	total: 2.19s	remaining: 305ms
878:	learn: 4727.0996183	total: 2.19s	remaining: 302ms
879:	learn: 4726.4523753	total: 2.2s	remaining: 300ms
880:	learn: 4725.7565012	total: 2.2s	remaining: 297ms
881:	learn: 

In [8]:
print(model.get_best_score())

{'learn': {'R2': 0.4649484007716691, 'RMSE': 4645.440600524081}}


## Cross validation 

In [9]:
model = CatBoostRegressor()
model.fit(X_train, y_train)
cross_val_score = cross_val_score(model, X_test, y_test, cv=10, scoring='r2').mean()

Learning rate set to 0.06138
0:	learn: 6297.4665022	total: 2.46ms	remaining: 2.46s
1:	learn: 6251.1235167	total: 4.63ms	remaining: 2.31s
2:	learn: 6209.9607147	total: 6.46ms	remaining: 2.15s
3:	learn: 6168.4693673	total: 8.3ms	remaining: 2.07s
4:	learn: 6134.5717157	total: 10.3ms	remaining: 2.05s
5:	learn: 6101.0595912	total: 12.7ms	remaining: 2.1s
6:	learn: 6070.5810388	total: 15.8ms	remaining: 2.24s
7:	learn: 6044.4797000	total: 18.2ms	remaining: 2.26s
8:	learn: 6019.4230880	total: 20.8ms	remaining: 2.29s
9:	learn: 6000.1576138	total: 23.4ms	remaining: 2.32s
10:	learn: 5978.9895762	total: 25.7ms	remaining: 2.31s
11:	learn: 5955.7379642	total: 28.2ms	remaining: 2.33s
12:	learn: 5937.9199704	total: 30.4ms	remaining: 2.31s
13:	learn: 5921.1820425	total: 33.2ms	remaining: 2.34s
14:	learn: 5906.4662959	total: 35.4ms	remaining: 2.33s
15:	learn: 5890.5422178	total: 37.8ms	remaining: 2.33s
16:	learn: 5876.9171295	total: 40.1ms	remaining: 2.32s
17:	learn: 5864.7206125	total: 42.8ms	remaining:

158:	learn: 5457.7755014	total: 379ms	remaining: 2s
159:	learn: 5456.3305287	total: 381ms	remaining: 2s
160:	learn: 5455.0829233	total: 383ms	remaining: 2s
161:	learn: 5453.9078249	total: 385ms	remaining: 1.99s
162:	learn: 5452.0554426	total: 389ms	remaining: 2s
163:	learn: 5449.6771977	total: 392ms	remaining: 2s
164:	learn: 5448.6305363	total: 395ms	remaining: 2s
165:	learn: 5446.3289842	total: 397ms	remaining: 2s
166:	learn: 5444.9651477	total: 400ms	remaining: 1.99s
167:	learn: 5442.6264190	total: 404ms	remaining: 2s
168:	learn: 5441.3296702	total: 406ms	remaining: 2s
169:	learn: 5439.2390020	total: 409ms	remaining: 2s
170:	learn: 5438.3290079	total: 411ms	remaining: 1.99s
171:	learn: 5435.8556678	total: 413ms	remaining: 1.99s
172:	learn: 5434.2596959	total: 416ms	remaining: 1.99s
173:	learn: 5433.1654326	total: 419ms	remaining: 1.99s
174:	learn: 5431.8941235	total: 421ms	remaining: 1.98s
175:	learn: 5430.5987769	total: 423ms	remaining: 1.98s
176:	learn: 5429.7418066	total: 425ms	re

321:	learn: 5223.5160741	total: 769ms	remaining: 1.62s
322:	learn: 5222.5044988	total: 771ms	remaining: 1.62s
323:	learn: 5221.3401228	total: 773ms	remaining: 1.61s
324:	learn: 5219.9284742	total: 776ms	remaining: 1.61s
325:	learn: 5219.2212000	total: 778ms	remaining: 1.61s
326:	learn: 5218.3020486	total: 781ms	remaining: 1.61s
327:	learn: 5217.4885255	total: 783ms	remaining: 1.6s
328:	learn: 5216.6139445	total: 786ms	remaining: 1.6s
329:	learn: 5215.5207026	total: 789ms	remaining: 1.6s
330:	learn: 5215.0314586	total: 791ms	remaining: 1.6s
331:	learn: 5214.4458356	total: 793ms	remaining: 1.6s
332:	learn: 5213.3547814	total: 796ms	remaining: 1.59s
333:	learn: 5211.9872336	total: 799ms	remaining: 1.59s
334:	learn: 5210.9020050	total: 801ms	remaining: 1.59s
335:	learn: 5210.0791350	total: 803ms	remaining: 1.59s
336:	learn: 5209.0823338	total: 806ms	remaining: 1.58s
337:	learn: 5207.3692868	total: 808ms	remaining: 1.58s
338:	learn: 5206.3743125	total: 811ms	remaining: 1.58s
339:	learn: 520

486:	learn: 5051.4348194	total: 1.16s	remaining: 1.22s
487:	learn: 5050.3170934	total: 1.16s	remaining: 1.22s
488:	learn: 5049.4756465	total: 1.17s	remaining: 1.22s
489:	learn: 5048.1667649	total: 1.17s	remaining: 1.22s
490:	learn: 5047.4598870	total: 1.17s	remaining: 1.21s
491:	learn: 5046.2196628	total: 1.17s	remaining: 1.21s
492:	learn: 5044.9279764	total: 1.18s	remaining: 1.21s
493:	learn: 5043.9977293	total: 1.18s	remaining: 1.21s
494:	learn: 5042.7776525	total: 1.18s	remaining: 1.21s
495:	learn: 5041.9948956	total: 1.18s	remaining: 1.2s
496:	learn: 5040.9462211	total: 1.19s	remaining: 1.2s
497:	learn: 5040.7092701	total: 1.19s	remaining: 1.2s
498:	learn: 5039.3445589	total: 1.19s	remaining: 1.2s
499:	learn: 5038.8354246	total: 1.19s	remaining: 1.19s
500:	learn: 5037.6762509	total: 1.2s	remaining: 1.19s
501:	learn: 5036.4534041	total: 1.2s	remaining: 1.19s
502:	learn: 5035.0088479	total: 1.2s	remaining: 1.19s
503:	learn: 5033.9811891	total: 1.2s	remaining: 1.19s
504:	learn: 5032.8

650:	learn: 4904.7470823	total: 1.55s	remaining: 831ms
651:	learn: 4903.8419195	total: 1.55s	remaining: 829ms
652:	learn: 4903.0852853	total: 1.55s	remaining: 827ms
653:	learn: 4902.1365133	total: 1.56s	remaining: 824ms
654:	learn: 4901.6147112	total: 1.56s	remaining: 822ms
655:	learn: 4900.4424134	total: 1.56s	remaining: 820ms
656:	learn: 4899.0809572	total: 1.56s	remaining: 818ms
657:	learn: 4898.4764603	total: 1.57s	remaining: 815ms
658:	learn: 4897.6555439	total: 1.57s	remaining: 813ms
659:	learn: 4896.9142290	total: 1.57s	remaining: 810ms
660:	learn: 4895.9325588	total: 1.57s	remaining: 808ms
661:	learn: 4895.1986567	total: 1.58s	remaining: 806ms
662:	learn: 4893.7904193	total: 1.58s	remaining: 804ms
663:	learn: 4893.2727051	total: 1.58s	remaining: 801ms
664:	learn: 4892.5659309	total: 1.58s	remaining: 799ms
665:	learn: 4891.8708550	total: 1.59s	remaining: 796ms
666:	learn: 4890.7749367	total: 1.59s	remaining: 794ms
667:	learn: 4889.6616644	total: 1.59s	remaining: 791ms
668:	learn

...

146:	learn: 5440.0442389	total: 255ms	remaining: 1.48s
147:	learn: 5437.5771049	total: 257ms	remaining: 1.48s
148:	learn: 5433.1229454	total: 259ms	remaining: 1.48s
149:	learn: 5431.0453942	total: 260ms	remaining: 1.48s
150:	learn: 5429.5995507	total: 262ms	remaining: 1.47s
151:	learn: 5426.7871348	total: 264ms	remaining: 1.47s
152:	learn: 5422.6530691	total: 266ms	remaining: 1.47s
153:	learn: 5418.6548479	total: 268ms	remaining: 1.47s
154:	learn: 5416.5494995	total: 270ms	remaining: 1.47s
155:	learn: 5413.3441305	total: 271ms	remaining: 1.47s
156:	learn: 5410.1098416	total: 273ms	remaining: 1.47s
157:	learn: 5407.1413228	total: 275ms	remaining: 1.46s
158:	learn: 5404.0285327	total: 276ms	remaining: 1.46s
159:	learn: 5400.8703618	total: 278ms	remaining: 1.46s
160:	learn: 5399.2877022	total: 280ms	remaining: 1.46s
161:	learn: 5398.1030579	total: 282ms	remaining: 1.46s
162:	learn: 5395.5020535	total: 283ms	remaining: 1.45s
163:	learn: 5393.3660901	total: 285ms	remaining: 1.45s
164:	learn

296:	learn: 5076.8079298	total: 516ms	remaining: 1.22s
297:	learn: 5076.4695362	total: 518ms	remaining: 1.22s
298:	learn: 5074.9881179	total: 520ms	remaining: 1.22s
299:	learn: 5073.0137535	total: 521ms	remaining: 1.22s
300:	learn: 5070.8032308	total: 523ms	remaining: 1.21s
301:	learn: 5068.9172251	total: 525ms	remaining: 1.21s
302:	learn: 5066.2891752	total: 527ms	remaining: 1.21s
303:	learn: 5064.6136862	total: 528ms	remaining: 1.21s
304:	learn: 5062.6951671	total: 531ms	remaining: 1.21s
305:	learn: 5061.5925173	total: 532ms	remaining: 1.21s
306:	learn: 5060.0895209	total: 534ms	remaining: 1.21s
307:	learn: 5058.9214536	total: 536ms	remaining: 1.2s
308:	learn: 5057.8331869	total: 538ms	remaining: 1.2s
309:	learn: 5055.6351433	total: 540ms	remaining: 1.2s
310:	learn: 5053.9046945	total: 541ms	remaining: 1.2s
311:	learn: 5051.7375657	total: 543ms	remaining: 1.2s
312:	learn: 5049.7998535	total: 545ms	remaining: 1.2s
313:	learn: 5049.4976213	total: 546ms	remaining: 1.19s
314:	learn: 5047

479:	learn: 4786.5975656	total: 836ms	remaining: 906ms
480:	learn: 4785.4380286	total: 838ms	remaining: 904ms
481:	learn: 4785.2943048	total: 839ms	remaining: 902ms
482:	learn: 4783.5272950	total: 841ms	remaining: 900ms
483:	learn: 4782.1747402	total: 843ms	remaining: 898ms
484:	learn: 4780.6179513	total: 845ms	remaining: 897ms
485:	learn: 4778.7967650	total: 846ms	remaining: 895ms
486:	learn: 4777.0978907	total: 848ms	remaining: 894ms
487:	learn: 4775.5149438	total: 850ms	remaining: 892ms
488:	learn: 4773.9448241	total: 852ms	remaining: 890ms
489:	learn: 4772.2899662	total: 853ms	remaining: 888ms
490:	learn: 4770.8093917	total: 855ms	remaining: 887ms
491:	learn: 4769.3097421	total: 857ms	remaining: 885ms
492:	learn: 4767.8968657	total: 859ms	remaining: 883ms
493:	learn: 4766.5270249	total: 861ms	remaining: 882ms
494:	learn: 4764.5062916	total: 862ms	remaining: 880ms
495:	learn: 4762.6219032	total: 864ms	remaining: 878ms
496:	learn: 4761.4410328	total: 866ms	remaining: 877ms
497:	learn

674:	learn: 4515.2618772	total: 1.22s	remaining: 589ms
675:	learn: 4513.9008691	total: 1.23s	remaining: 587ms
676:	learn: 4512.6790300	total: 1.23s	remaining: 586ms
677:	learn: 4510.8773049	total: 1.23s	remaining: 584ms
678:	learn: 4509.5921469	total: 1.23s	remaining: 582ms
679:	learn: 4507.2126407	total: 1.23s	remaining: 581ms
680:	learn: 4505.9973646	total: 1.24s	remaining: 579ms
681:	learn: 4505.0489378	total: 1.24s	remaining: 577ms
682:	learn: 4504.0795817	total: 1.24s	remaining: 576ms
683:	learn: 4502.6891195	total: 1.24s	remaining: 574ms
684:	learn: 4501.7881461	total: 1.24s	remaining: 572ms
685:	learn: 4500.9906330	total: 1.25s	remaining: 570ms
686:	learn: 4499.9383755	total: 1.25s	remaining: 569ms
687:	learn: 4498.8151592	total: 1.25s	remaining: 567ms
688:	learn: 4497.3883243	total: 1.25s	remaining: 565ms
689:	learn: 4495.5743861	total: 1.25s	remaining: 563ms
690:	learn: 4493.5332178	total: 1.25s	remaining: 562ms
691:	learn: 4491.9586535	total: 1.26s	remaining: 560ms
692:	learn

869:	learn: 4288.7063197	total: 1.59s	remaining: 238ms
870:	learn: 4288.3302318	total: 1.59s	remaining: 236ms
871:	learn: 4287.2440524	total: 1.59s	remaining: 234ms
872:	learn: 4286.1299627	total: 1.6s	remaining: 232ms
873:	learn: 4284.8111323	total: 1.6s	remaining: 230ms
874:	learn: 4283.8328948	total: 1.6s	remaining: 229ms
875:	learn: 4283.1711209	total: 1.6s	remaining: 227ms
876:	learn: 4282.3050498	total: 1.61s	remaining: 225ms
877:	learn: 4281.2846952	total: 1.61s	remaining: 224ms
878:	learn: 4281.1766308	total: 1.61s	remaining: 222ms
879:	learn: 4279.9867840	total: 1.61s	remaining: 220ms
880:	learn: 4278.1589647	total: 1.61s	remaining: 218ms
881:	learn: 4276.9065001	total: 1.62s	remaining: 216ms
882:	learn: 4275.7372214	total: 1.62s	remaining: 214ms
883:	learn: 4274.9720522	total: 1.62s	remaining: 213ms
884:	learn: 4273.1165838	total: 1.62s	remaining: 211ms
885:	learn: 4272.3829575	total: 1.62s	remaining: 209ms
886:	learn: 4270.8263305	total: 1.63s	remaining: 207ms
887:	learn: 42

In [10]:
cross_val_score

0.1837855667471339